In [70]:
import pandas as pd
import numpy as np

In [248]:
user_segment = pd.read_csv('output/tnse_data2.csv')
user_segment['LFMStr'] = user_segment['LFMScore'].apply(lambda x: "-".join(list(str(x)) ))

port_value = pd.read_csv('data/portValue.csv')
port_setting = pd.read_csv('data/portSetting.csv')

old_demographic = pd.read_csv('data/rawDataUser.csv')
del old_demographic['_id']

In [296]:
location = pd.read_csv('data/TH.txt', delimiter="\t", header=None)[[0,1,2,3]]
location.columns = ["country", "post_code", "district", "province"]
location = location.drop_duplicates(subset=['post_code'], keep='first', inplace=False)
location.head()

,country,post_code,district,province
0,TH,10100,Pom Prap Sattru Phai,Bangkok
2,TH,10110,Wattana,Bangkok
3,TH,10120,Yannawa,Bangkok
5,TH,10140,Rat Burana,Bangkok
7,TH,10150,Bang Khun Thian,Bangkok


In [250]:
old_demographic.head()

,accountNo,phoneNo,riskLevel,questionRisk/sumPointRisk,questionRisk/invesmentForeign,questionRisk/lossAllMoney,questionRisk/loss,questionRisk/lossMoneyCantAccept,questionRisk/highRiskHighReturn,questionRisk/invesmentType,...,questionaire/birthDate,account/gender,account/residenceRegisterationAddress/country,account/residenceRegisterationAddress/postalCode,account/residenceRegisterationAddress/province,account/residenceRegisterationAddress/district,account/residenceRegisterationAddress/subDistrict,birthDate/day,birthDate/month,birthDate/year
0,ODI0000034,099-241-5419,5.0,39.0,1,1.0,4.0,4.0,4.0,4.0,...,1978-11-15T17:00:00.000Z,Male,ประเทศไทย,10160,กรุงเทพมหานคร,บางแค,บางแค,NaN,NaN,NaN
1,ODI0000059,091-779-8273,5.0,38.0,1,1.0,4.0,4.0,4.0,3.0,...,1995-08-23T17:00:00.000Z,Male,ประเทศไทย,11000,นนทบุรี,เมืองนนทบุรี,บางรักน้อย,NaN,NaN,NaN
2,ODI0000067,098-556-6555,4.0,22.0,1,1.0,1.0,1.0,1.0,2.0,...,1983-09-09T17:00:00.000Z,Female,ประเทศไทย,10120,กรุงเทพมหานคร,สาทร,ทุ่งวัดดอน,NaN,NaN,NaN
3,ODI0000068,081-838-3333,5.0,37.0,1,1.0,4.0,4.0,4.0,4.0,...,1986-07-21T17:00:00.000Z,Male,ประเทศไทย,10300,กรุงเทพมหานคร,ดุสิต,สวนจิตรลดา,NaN,NaN,NaN
4,ODI0000072,097-195-8555,4.0,34.0,1,1.0,4.0,4.0,3.0,4.0,...,1980-03-07T17:00:00.000Z,Male,ประเทศไทย,10250,กรุงเทพมหานคร,สวนหลวง,สวนหลวง,NaN,NaN,NaN


In [251]:
use_demographic = old_demographic[['accountNo','phoneNo', 'questionaire/birthDate','birthDate/year','account/gender','account/residenceRegisterationAddress/postalCode', 'riskLevel']]
use_demographic.head()

,accountNo,phoneNo,questionaire/birthDate,birthDate/year,account/gender,account/residenceRegisterationAddress/postalCode,riskLevel
0,ODI0000034,099-241-5419,1978-11-15T17:00:00.000Z,NaN,Male,10160,5.0
1,ODI0000059,091-779-8273,1995-08-23T17:00:00.000Z,NaN,Male,11000,5.0
2,ODI0000067,098-556-6555,1983-09-09T17:00:00.000Z,NaN,Female,10120,4.0
3,ODI0000068,081-838-3333,1986-07-21T17:00:00.000Z,NaN,Male,10300,5.0
4,ODI0000072,097-195-8555,1980-03-07T17:00:00.000Z,NaN,Male,10250,4.0


หลักของเบอร์สวย
- ตอง
- เรียง 3 ตัวขึ้นไป เช่น 123,456,567

In [252]:
import re 
from datetime import datetime

def goodTelNo(x):
    tong = ['111','222','333','444','555','666','777','888','999']
    rieng = ['012','123','234','345','456','567','678','789','890']
    for ind in range(len(tong)):
        if (tong[ind] in x) or (rieng[ind] in x):
            return "Good Number"
    return "Normal Number"

def age(x,y):
    x = str(x)
    if len(x) > 3:
        year = x[0:4]
    else:
        year = y
    
    age_num = 2019-int(year)
    if 20 <= age_num <= 24:
        return '20-24'
    elif 25 <= age_num <= 34:
        return '25-34'
    elif 35 <= age_num <= 44:
        return '35-44'
    elif 45 <= age_num :
        return '45+'
    else:
        return age_num

def provinceLocation(post_code):
    return str(post_code)[0:2]

def income(x,y):
    x = str(x)
    if len(x) > 3:
        year = x[0:4]
    else:
        year = y
    
    age_num = 2019-int(year)
    if 20 <= age_num <= 24:
        return '20-24'
    elif 25 <= age_num <= 34:
        return '25-34'
    elif 35 <= age_num <= 44:
        return '35-44'
    elif 45 <= age_num :
        return '45+'
    else:
        return age_num

In [253]:
demo = pd.DataFrame({'acc_no': use_demographic['accountNo'],
                     'tel': use_demographic['phoneNo'].apply(goodTelNo),
                     'age': use_demographic.apply(lambda row: age(row['questionaire/birthDate'], row['birthDate/year']), axis=1),
                     'gender': use_demographic['account/gender'],
                     'post_code': use_demographic['account/residenceRegisterationAddress/postalCode'],
                     'prov_code': use_demographic['account/residenceRegisterationAddress/postalCode'].apply(provinceLocation),
                     'risk_level': use_demographic['riskLevel'],
                    })

In [254]:
demo.head()

,acc_no,tel,age,gender,post_code,prov_code,risk_level
0,ODI0000034,Normal Number,35-44,Male,10160,10,5.0
1,ODI0000059,Normal Number,20-24,Male,11000,11,5.0
2,ODI0000067,Good Number,35-44,Female,10120,10,4.0
3,ODI0000068,Good Number,25-34,Male,10300,10,5.0
4,ODI0000072,Good Number,35-44,Male,10250,10,4.0


In [255]:
demo.shape

(3970, 7)

In [256]:
user_111 = user_segment[user_segment['LFMScore'] == 111]
user_444 = user_segment[user_segment['LFMScore'] == 444]
user_segment.shape, user_111.shape, user_444.shape

((1833, 8), (266, 8), (113, 8))

In [304]:
segment_and_demo = pd.merge(user_segment ,demo,how='left', on='acc_no')
lastMonthRiskLevel = port_setting[port_setting['portNo'] == 'RB01'][['riskLevel','accNo']].copy()
lastMonthRiskLevel['last_risk_level'] = lastMonthRiskLevel['riskLevel']
del lastMonthRiskLevel['riskLevel']

all_merge = pd.merge(segment_and_demo, location,how='left', on='post_code')
all_merge2 = pd.merge(all_merge, lastMonthRiskLevel,how='left', left_on='acc_no', right_on='accNo')

In [305]:
all_merge2.to_csv('output/all_merge.csv', index=False)

In [306]:
all_merge2

,acc_no,LFMScore,L_quartile,F_quartile,M_quartile,tnse-x,tnse-y,LFMStr,tel,age,gender,post_code,prov_code,risk_level,country,district,province,accNo,last_risk_level
0,ODI0000034,444,4 : มากกว่า 135 วัน,4 : มากกว่า 5 ครั้ง,"4 : มากกว่า 15,261.8 บาท",282.161530,-23.804598,4-4-4,Normal Number,35-44,Male,10160.0,10,5.0,TH,Phasi Charoen,Bangkok,ODI0000034,5.0
1,ODI0000059,441,4 : มากกว่า 135 วัน,4 : มากกว่า 5 ครั้ง,"1 : ไม่เกิน 1,000 บาท",139.034910,-285.442570,4-4-1,Normal Number,20-24,Male,11000.0,11,5.0,TH,Muang Nonthaburi,Nonthaburi,ODI0000059,4.0
2,ODI0000067,444,4 : มากกว่า 135 วัน,4 : มากกว่า 5 ครั้ง,"4 : มากกว่า 15,261.8 บาท",259.053700,-27.004026,4-4-4,Good Number,35-44,Female,10120.0,10,4.0,TH,Yannawa,Bangkok,ODI0000067,4.0
3,ODI0000068,434,4 : มากกว่า 135 วัน,3 : 3-5 ครั้ง,"4 : มากกว่า 15,261.8 บาท",141.506680,-0.824704,4-3-4,Good Number,25-34,Male,10300.0,10,5.0,TH,Dusit,Bangkok,ODI0000068,5.0
4,ODI0000072,443,4 : มากกว่า 135 วัน,4 : มากกว่า 5 ครั้ง,"3 : 7,001-15,261.8 บาท",239.113050,-191.500950,4-4-3,Good Number,35-44,Male,10250.0,10,4.0,TH,Suang Luang,Bangkok,ODI0000072,5.0
5,ODI0000080,411,4 : มากกว่า 135 วัน,1 : 1 ครั้ง,"1 : ไม่เกิน 1,000 บาท",-44.943480,-340.739470,4-1-1,Normal Number,25-34,Female,10230.0,10,4.0,TH,Lat Phrao,Bangkok,ODI0000080,4.0
6,ODI0000082,424,4 : มากกว่า 135 วัน,2 : 2 ครั้ง,"4 : มากกว่า 15,261.8 บาท",105.755340,-55.082027,4-2-4,Normal Number,25-34,Female,74130.0,74,5.0,NaN,NaN,NaN,ODI0000082,5.0
7,ODI0000083,411,4 : มากกว่า 135 วัน,1 : 1 ครั้ง,"1 : ไม่เกิน 1,000 บาท",-44.943480,-340.739470,4-1-1,Normal Number,35-44,Male,55120.0,55,3.0,TH,Pua,Nan,ODI0000083,4.0
8,ODI0000084,111,1 : ไม่เกิน 37 วัน,1 : 1 ครั้ง,"1 : ไม่เกิน 1,000 บาท",-229.231870,34.217587,1-1-1,Good Number,45+,Female,10900.0,10,3.0,NaN,NaN,NaN,ODI0000084,3.0
9,ODI0000086,422,4 : มากกว่า 135 วัน,2 : 2 ครั้ง,"2 : 1,001-7,000 บาท",-17.013250,-247.526170,4-2-2,Normal Number,35-44,Female,60000.0,60,3.0,TH,Muang Nakhon Sawan,Nakhon Sawan,ODI0000086,3.0
